In [1]:
import os
import csv
import pandas as pd
from torchvision import transforms
from PIL import Image

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("data_pre_processing").getOrCreate()


In [6]:
import os
os.chdir("/home/hduser/Descargas/weather")


In [7]:
folder = "/home/hduser/Descargas/weather"
#directory with images folder

# Data preprocessing

In [8]:
# create CSV with labels and image names

# assign label
def assign_label(nome_file):
    if nome_file.startswith("rain"):
        return "rain"
    elif nome_file.startswith("cloudy"):
        return "cloudy"
    elif nome_file.startswith("shine"):
        return "shine"
    elif nome_file.startswith("sunrise"):
        return "sunrise"
    else:
        return "other"

# List
dati = []


for x in os.listdir(folder):
    if x.endswith(tuple(".jpg")):
        label = assign_label(x)
        dati.append((x, label))


df = pd.DataFrame(dati, columns=["image", "label"])



In [9]:
df.head()

,image,label
0,rain108.jpg,rain
1,shine24.jpg,shine
2,shine94.jpg,shine
3,shine231.jpg,shine
4,sunrise334.jpg,sunrise


In [10]:
# input dimensions for the CNN
input_dim = (224,224)
input_dir_cnn = folder + "/images/input_images_cnn"


In [11]:
os.makedirs(input_dir_cnn, exist_ok = True)


In [12]:
transformation_for_cnn_input = transforms.Compose([transforms.Resize(input_dim)])


In [15]:
output_dir = 'Descargas/output_images_cnn'


In [16]:
folder = "/home/hduser/Descargas/weather" 
input_dir_cnn = output_dir  


os.makedirs(input_dir_cnn, exist_ok=True)

for item in os.listdir(folder):
    item_path = os.path.join(folder, item)

    
    if os.path.isfile(item_path):  # if is a file
        I = Image.open(item_path)

        
        if I.mode != 'RGB': # convert in RGB (colors)
            I = I.convert('RGB')

        
        newI = transformation_for_cnn_input(I) # transform for cnn

        
        newI.save(os.path.join(input_dir_cnn, item)) # save in jpeg

        # close images
        I.close()
        newI.close()


In [17]:
import torch
from torchvision import models
from PIL import Image

In [18]:
class Img2VecResnet18():
    
    def __init__(self):
        
        self.device = torch.device("cpu")
        self.numberFeatures = 512
        self.modelName = "resnet-18"
        self.model, self.featureLayer = self.getFeatureLayer() 
        self.model = self.model.to(self.device) 
        self.model.eval()
        self.toTensor = transforms.ToTensor()
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])
        
    def getFeatureLayer(self):
    
        cnnModel = models.resnet18(pretrained=True) 
        layer = cnnModel._modules.get('avgpool') 
        self.layer_output_size = 512
        return cnnModel, layer

    def getVec(self, img):
        image = self.normalize(self.toTensor(img)).unsqueeze(0).to(self.device) 
        embedding = torch.zeros(1, self.numberFeatures, 1, 1)
        def copyData(m, i, o): embedding.copy_(o.data)
        h = self.featureLayer.register_forward_hook(copyData)
        self.model(image)
        h.remove()
        return embedding.numpy()[0, :, 0, 0]


    


In [21]:
import tqdm
from torchvision import models, transforms 

In [22]:
img2vec = Img2VecResnet18()

/home/hduser/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hduser/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/hduser/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


In [23]:
allVectors = {}

In [24]:
for image_name in os.listdir(input_dir_cnn):
    I = Image.open(os.path.join(input_dir_cnn, image_name))
    vec = img2vec.getVec(I)
    allVectors[image_name] = vec
    I.close()

In [25]:
data_folder = os.path.expanduser('~/Descargas')

In [26]:
pd.DataFrame(allVectors).transpose().to_csv(data_folder+'/input_data_vectors.csv')


In [27]:
spark.stop()

In [29]:
import warnings
warnings.filterwarnings("ignore")

In [30]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

In [32]:
spark = SparkSession.builder.appName("model").getOrCreate()

2023-09-24 22:26:50,340 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2023-09-24 22:26:50,340 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2023-09-24 22:26:50,340 WARN util.Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [33]:
input_df = spark.read.option('inferschema',True).\
                    csv('hdfs://localhost:9000/user1/input_data_vectors.csv')
                                     

In [34]:
input_df.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14',
 '_c15',
 '_c16',
 '_c17',
 '_c18',
 '_c19',
 '_c20',
 '_c21',
 '_c22',
 '_c23',
 '_c24',
 '_c25',
 '_c26',
 '_c27',
 '_c28',
 '_c29',
 '_c30',
 '_c31',
 '_c32',
 '_c33',
 '_c34',
 '_c35',
 '_c36',
 '_c37',
 '_c38',
 '_c39',
 '_c40',
 '_c41',
 '_c42',
 '_c43',
 '_c44',
 '_c45',
 '_c46',
 '_c47',
 '_c48',
 '_c49',
 '_c50',
 '_c51',
 '_c52',
 '_c53',
 '_c54',
 '_c55',
 '_c56',
 '_c57',
 '_c58',
 '_c59',
 '_c60',
 '_c61',
 '_c62',
 '_c63',
 '_c64',
 '_c65',
 '_c66',
 '_c67',
 '_c68',
 '_c69',
 '_c70',
 '_c71',
 '_c72',
 '_c73',
 '_c74',
 '_c75',
 '_c76',
 '_c77',
 '_c78',
 '_c79',
 '_c80',
 '_c81',
 '_c82',
 '_c83',
 '_c84',
 '_c85',
 '_c86',
 '_c87',
 '_c88',
 '_c89',
 '_c90',
 '_c91',
 '_c92',
 '_c93',
 '_c94',
 '_c95',
 '_c96',
 '_c97',
 '_c98',
 '_c99',
 '_c100',
 '_c101',
 '_c102',
 '_c103',
 '_c104',
 '_c105',
 '_c106',
 '_c107',
 '_c108',
 '_c109',
 '_c110',


In [35]:
from pyspark.ml.feature import VectorAssembler

In [36]:
vector_columns = input_df.columns[1:]

In [37]:
assembler = VectorAssembler(inputCols = vector_columns, outputCol = "features")

In [38]:
output = assembler.transform(input_df)

In [39]:
output = output.select('_c0','features')

In [40]:
output.show(3, vertical = True)

2023-09-24 22:28:07,374 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0------------------------
 _c0      | null                 
 features | [0.0,1.0,2.0,3.0,... 
-RECORD 1------------------------
 _c0      | cloudy126.jpg        
 features | [0.20776045,0.534... 
-RECORD 2------------------------
 _c0      | rain20.jpg           
 features | [0.07726963,1.333... 
only showing top 3 rows



In [41]:
output.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- features: vector (nullable = true)

